## Introduction
A regime analysis is significant in asset allocation and asset-liability management for
long-term investors because of the contagion and related effects during crash periods: the
correlation between risky assets and volatility will greatly increase during the crash
periods, thus creating severe difficulty in risk management and protecting investor capital
with traditional portfolio models.

Many existing methods are based on econometric models which assume a fixed structural model. However,
financial return or macroeconomic data tends to be noisy and affected by myriad of
factors. The state-of-the-art approach discussed in Mulvey and Liu (2016), trend-filtering,
is non-parametric, data-driven and model-free. The algorithm was first introduced by
Kim et al. (2009) and generalised in Tibshirani (2014).

In this part, we will implement the algorithm to obtain regimes of U.S. equity. First, we need the relevant packages and data.

In [18]:
import numpy as np #for numerical array data
import pandas as pd #for tabular data
from scipy.optimize import minimize
from scipy.optimize import linprog
import matplotlib.pyplot as plt #for plotting purposes
import matplotlib
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
import cvxpy as cp

In [3]:
# Import Data
SP500_data = pd.read_csv('datasets/SP500.csv')
SP500_data.head()

,DATE,SP500,SP500TR
0,1/1/85,0.5585,84.7932
1,2/1/85,7.4747,91.1312
2,3/1/85,5.5791,96.2155
3,4/1/85,-1.1260,95.1321
4,5/1/85,5.3201,100.1932


In the trend-filtering algorithm, we manage to find some ‘fitted’ time series that serves as the signal of the trend. This new time series can be obtained by solving the following optimizationproblem:

\begin{equation*} 
    \hat{\beta} = \text{argmin}_{\beta \in \mathbb{R}^n} ||x-\beta||_2^2 + \lambda||D\beta||_1 .
\end{equation*}

where

\begin{equation*} 
D =
    \begin{bmatrix}
       1 & -1 & 0 & \dots & 0 & 0  \\
       0  & 1 &-1 & \dots & 0 & 0 \\
       \vdots \\
       0  & 0 & 0 & \dots & -1 & 0\\
       0  & 0 & 0 & \dots & 1 & -1
    \end{bmatrix}
\in \mathbb{R}^{(n-1)\times n}.
\end{equation*}

In [10]:
ret = SP500_data.as_matrix(columns=['SP500']) # Percentage
n = np.size(ret)
x_ret = ret.reshape(n)

Dfull = np.diag([1]*n) - np.diag([1]*(n-1),1)
D = Dfull[0:(n-1),]

In [14]:
# Scipy Implementation - satisfactory
lamb = 16
def trend_filtering(x):
    return np.sum((x_ret-x)**2) + lamb * np.sum(np.abs(x[1:]-x[:-1]))

res = minimize(trend_filtering, ret/2, method="SLSQP",tol=1e-6,options={'maxiter':10000})
res.fun

6319.379923850933

In [16]:
# CVXPY Implementation
beta = cp.Variable(n)
lambd = cp.Parameter(nonneg=True)
x_ret = ret.reshape(n)

def tf_obj(x,beta,lambd):
    return cp.norm(x-beta,2)**2 + lambd*cp.norm(cp.matmul(D, beta),1)

problem = cp.Problem(cp.Minimize(tf_obj(x_ret, beta, lambd)))

lambd.value = 16
problem.solve()

#beta.value

6318.804147421546

In [19]:
beta.value

array([ 1.96123279,  1.96123284,  1.96123267,  1.96122969,  1.96122948,
        1.81815318,  1.81814671,  1.81814671,  1.81814629,  1.81814647,
        1.81814648,  1.81814647,  1.81814663,  1.81814656,  1.8181465 ,
        1.81814651,  1.81814658,  1.81814652,  1.81814646,  1.81814647,
        1.81814658,  3.54667211,  3.54667293,  3.54667308,  3.54667317,
        3.5466713 ,  2.88524831,  2.76525802,  2.76525784,  2.76525776,
        2.76525755,  2.76525669, -2.19218787, -6.88901989, -6.8890192 ,
        1.63521965,  1.63521972,  1.63521969,  1.63521953,  1.63521956,
        1.6352196 ,  1.63521966,  1.63521962,  1.63521965,  1.63522001,
        1.63522013,  1.6352202 ,  1.63522047,  1.63522074,  1.63522072,
        1.63522088,  1.635221  ,  1.63522099,  1.6352209 ,  1.63522089,
        1.63521866,  0.68449028,  0.68448957,  0.68448943,  0.68448918,
        0.68448853,  0.68448856,  0.68448856,  0.6844885 ,  0.68448855,
        0.12765087,  0.12764984,  0.12764915,  0.12764924,  0.12

## Regime Identification

Once we obtain the "smoothed" betas, we will label the month "normal" for positive betas and "crash" for negative betas.

In [9]:
# [[To do]] Plot the log SP500 along with the regime
SP500 = SP500_data.as_matrix(columns=['SP500TR'])

### Some changes need to be applied to the plot:
1) Time period: 1985-2017 (see dataset)

2) Since the span of SP500 is large, use log series of SP500 in the plot (black line)

&nbsp; [<img src="regime_sample.jpg">]

In [ ]:
# [[To do]] Redo this with smaller values of lambda, obtaining more regime switches
lambd.value = 15 # Also try 14, 13, 12, 11 and 10

In [ ]:
# [[To do]] Maybe it is better to have a "Regularization path", i.e. number of regimes vs values of lambda. 

lambd_values = np.logspace(0, 4, 100)

beta_values = []
for v in lambd_values:
    lambd.value = v
    problem.solve()

    beta_values.append(beta.value)
